In [82]:
import pickle
import os
import pandas as pd
from glob import glob
from tqdm import tqdm

In [16]:
# # 删除问题行
# metadata_path = "/data_nas/GoogleSV/metadata/China/HongKong/pano_2024-08-17 13:11:09.125553_23512.p"
# metadata = pd.read_pickle(metadata_path)
# error_line = metadata[metadata['panoid'] == "AF1QipOEQeUmG17kqli1N84XJmoOdliKcXrxp64sheE6"]
# metadata = metadata.drop(error_line.index).reset_index(drop=True)
# metadata.to_pickle(metadata_path)

In [17]:
dataset_path = "/data1/cehou_data/LLM_safety/img_text_data/baseline"
dataset = os.listdir(dataset_path)

In [18]:
glob(f'{dataset_path}/dataset_baseline_*')

['/data1/cehou_data/LLM_safety/img_text_data/baseline/dataset_baseline_baseline_baseline_baseline_1401.pkl',
 '/data1/cehou_data/LLM_safety/img_text_data/baseline/dataset_baseline_baseline_baseline_baseline_1400.pkl',
 '/data1/cehou_data/LLM_safety/img_text_data/baseline/dataset_baseline_baseline_baseline_baseline_PlacePulse_2400.pkl',
 '/data1/cehou_data/LLM_safety/img_text_data/baseline/dataset_baseline_baseline_baseline_baseline_PlacePulse_3850.pkl',
 '/data1/cehou_data/LLM_safety/img_text_data/baseline/dataset_baseline_baseline_baseline_baseline_501.pkl']

In [19]:
for i,file in enumerate(glob(f'{dataset_path}/dataset_baseline_*')):
    with open(file, "rb") as f:
        data = pickle.load(f)
        # print(len(data))
        if i == 0:
            all_data = data
        else:
            all_data.extend(data)
print(f"total data number: {len(all_data)}")

total data number: 5252


In [20]:
# 使用字典来删除重复项，同时保持顺序
unique_all_data = list({item['GSV_idx']: item for item in all_data}.values())
# all_data = unique_all_data
print(f"Number of unique items in all_data: {len(unique_all_data)}")
all_data_df = pd.DataFrame(unique_all_data)

Number of unique items in all_data: 3851


In [77]:
safety_perception = pd.read_csv("/data_nas/cehou/LLM_safety/PlacePulse2.0/image_perception.csv")
safety_perception.rename(columns={'Image_ID': 'GSV_name'}, inplace=True)
# safety_perception = safety_perception.drop_duplicates(subset=['GSV_name']).reset_index(drop=True)

def retidy_oneline(GSV_id):
    aaa = safety_perception[safety_perception['GSV_name'] == GSV_id].T
    aaa.columns = aaa.iloc[1]
    aaa = aaa.iloc[2:,:]
    aaa['GSV_name'] = GSV_id
    return aaa

In [86]:
for i, line in tqdm(all_data_df.iterrows()):
    tem = retidy_oneline(line['GSV_name'])
    # safety_perception = safety_perception.merge(tem, how='left', on='GSV_name')
    if i == 0:
        all_df = tem
    else:
        all_df = pd.concat([all_df, tem], axis=0)


3851it [01:29, 43.16it/s]


In [89]:
all_data_df = all_data_df.merge(all_df, how='left', on='GSV_name')

In [95]:
all_data_df

,GSV_idx,GSV_name,GSV_path,text_description,age,gender,location,event,text_description_short,safety,lively,beautiful,wealthy,depressing,boring
0,501,50f42b7bfdc9f065f00013ba,/data_nas/cehou/LLM_safety/PlacePulse2.0/photo...,[INST] Please design a street safety perceptio...,baseline,baseline,baseline,baseline,NaN,0.357143,0.333333,0.333333,0.527778,0.527778,0.333333
1,502,50f42b7bfdc9f065f00013bb,/data_nas/cehou/LLM_safety/PlacePulse2.0/photo...,[INST] Please design a street safety perceptio...,baseline,baseline,baseline,baseline,NaN,0.385284,0.333333,0.417857,0.5,0.644444,0.400794
2,503,50f42b7bfdc9f065f00013bd,/data_nas/cehou/LLM_safety/PlacePulse2.0/photo...,[INST] Please design a street safety perceptio...,baseline,baseline,baseline,baseline,NaN,0.445268,0.544907,0.3,0.444444,0.444444,0.333333
3,504,50f42b7bfdc9f065f00013be,/data_nas/cehou/LLM_safety/PlacePulse2.0/photo...,[INST] Please design a street safety perceptio...,baseline,baseline,baseline,baseline,NaN,0.631217,0.333333,0.305556,0.548095,0.425926,0.666667
4,505,50f42b7bfdc9f065f00013c0,/data_nas/cehou/LLM_safety/PlacePulse2.0/photo...,[INST] Please design a street safety perceptio...,baseline,baseline,baseline,baseline,NaN,0.333333,0.426263,0.441667,0.487037,0.440476,0.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3846,3846,50f4352efdc9f065f0002c52,/data_nas/cehou/LLM_safety/PlacePulse2.0/photo...,[INST] Please design a street safety perceptio...,baseline,baseline,baseline,baseline,NaN,0.477143,0.666667,0.444444,0.527778,0.333333,0.333333
3847,3847,50f4352efdc9f065f0002c53,/data_nas/cehou/LLM_safety/PlacePulse2.0/photo...,[INST] Please design a street safety perceptio...,baseline,baseline,baseline,baseline,NaN,0.522576,0.337542,0.277778,0.377778,0.333333,0.455556
3848,3848,50f4352efdc9f065f0002c56,/data_nas/cehou/LLM_safety/PlacePulse2.0/photo...,[INST] Please design a street safety perceptio...,baseline,baseline,baseline,baseline,NaN,0.696868,0.423822,0.611111,0.402886,0.431481,0.333333
3849,3849,50f4352efdc9f065f0002c5c,/data_nas/cehou/LLM_safety/PlacePulse2.0/photo...,[INST] Please design a street safety perceptio...,baseline,baseline,baseline,baseline,NaN,0.62,0.508571,0.277778,0.438889,0.466667,0.444444


In [ ]:
def get_qa(text_description):
    answer = [i.split("<\\s> [INST]")[0] for i in text_description.split(" [/INST] ")][1:]
    question = [text_description.split(" [/INST] ")[0][7:]] + [i.split("<\\s> [INST]")[1] for i in text_description.split(" [/INST] ")[1:-1]]
    return answer, question

def text_processing(dataset, data_type):
    '''
    data_type: 'baseline' or 'others'
    example: text_processing("/data1/cehou_data/LLM_safety/img_text_data/dataset_baseline_baseline_baseline_baseline_1401.pkl", 'baseline')
    '''
    # dataset = pd.read_pickle(dataset)
    text_description = [dataset[i]['text_description'] for i in range(len(dataset))]

    answer_list = [get_qa(text_description[i])[0] for i in range(len(text_description))]
    question_list = [get_qa(text_description[i])[1] for i in range(len(text_description))]
    if data_type == 'baseline':
        text_description_new = [answer_list[i][1] for i in range(len(text_description))]
        for i in range(len(dataset)):
            dataset[i]['text_description_short'] = text_description_new[i]
    else:
        print('not implemented')

    # with open(dataset_path, 'wb') as f:
    #     pickle.dump(dataset, f)
    return None


In [93]:
data_config = dataset[0].split("_")
save_filename = f"/data1/cehou_data/LLM_safety/img_text_data/baseline/dataset_{data_config[1]}_{data_config[2]}_{data_config[3]}_{data_config[4]}_{len(all_data_df)}.pkl"
all_data_df.to_pickle(save_filename)
print(f"save the data into {save_filename}")

In [94]:
# data_config = dataset[0].split("_")
# save_filename = f"/data1/cehou_data/LLM_safety/img_text_data/dataset_{data_config[1]}_{data_config[2]}_{data_config[3]}_{data_config[4]}_{len(all_data)}.pkl"
# with open(save_filename, "wb") as f:
#     pickle.dump(all_data, f)
# print(f"save the data into {save_filename}")